# 项目 - 航空公司 AI 助手

现在我们将整合所学知识，为一家航空公司制作一个 AI 客服助手

In [ ]:
# 导入库

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# 初始化

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API 密钥存在，前缀为 {openai_api_key[:8]}")
else:
    print("未设置 OpenAI API 密钥")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# 另外，如果您想使用 Ollama 而不是 OpenAI
# 请检查 Ollama 是否在本地运行（参见 week1/day2 练习），然后取消下面两行的注释
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [ ]:
system_message = "你是一家名为 FlightAI 的航空公司的乐于助人的助手。 "
system_message += "请给出简短、礼貌的回答，不超过一句话。 "
system_message += "回答务必准确。如果不知道答案，请如实说明。"

In [ ]:
# 这个函数看起来比我视频中的那个要简单得多，因为我们利用了最新的 Gradio 更新

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

## 工具

工具是前沿 LLM 提供的一个极其强大的功能。

通过工具，你可以编写一个函数，并让 LLM 在其响应中调用该函数。

听起来有点神奇……我们是给了它在我们的机器上运行代码的权力吗？

嗯，差不多吧。

In [ ]:
# 让我们从创建一个有用的函数开始

ticket_prices = {"北京": "799", "上海": "899", "成都": "1400", "济南": "499"}

def get_ticket_price(destination_city):
    print(f"工具 get_ticket_price 已为 {destination_city} 调用")
    city = destination_city.lower()
    return ticket_prices.get(city, "未知")

In [ ]:
get_ticket_price("济南")

In [ ]:
# 描述我们的函数需要一个特定的字典结构：

price_function = {
    "name": "get_ticket_price",
    "description": "获取到目的地城市的往返机票价格。当您需要知道机票价格时（例如，当客户询问‘到这个城市的机票多少钱’时），请调用此函数",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "客户想要前往的城市",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# 然后将其包含在一个工具列表中：

tools = [{"type": "function", "function": price_function}]

## 让 OpenAI 使用我们的工具

要让 OpenAI “调用我们的工具”，需要一些繁琐的设置

我们实际做的是给 LLM 一个机会，让它通知我们它希望我们运行这个工具。

新的聊天函数看起来是这样的：

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# 我们必须编写 handle_tool_call 函数：

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()